In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/MyDrive/

In [3]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10948908292365568286
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9399762944
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15725146134698470536
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:22:00.0, compute capability: 7.5"
 xla_global_id: 416903419,
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 1745276110
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12549827829692726367
 physical_device_desc: "device: 1, name: NVIDIA RTX A2000 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 2144165316]

In [5]:
import pandas as pd
#import pandas_profiling
import numpy as np
import cupy as cp
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import roc_auc_score
from sklearn.tree import plot_tree
from sklearn.metrics import mean_squared_error

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import optuna.integration.lightgbm as lgb
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier


print("OK\n")

OK



In [6]:
app_train =  pd.read_csv('./home-credit-default-risk/exports/app_train_nn.csv')
app_test =  pd.read_csv('./home-credit-default-risk/exports/app_test_nn.csv')
# 他社債務
hc_br_train = pd.read_csv('./home-credit-default-risk/exports/hc_br_train_axis-60.csv')
hc_br_test = pd.read_csv('./home-credit-default-risk/exports/hc_br_test_axis-60.csv')
# hc Cash loans/Cash loans
hc_only_cash_cash_train = pd.read_csv('./home-credit-default-risk/exports/hc_only-cash-cash_train_axis-24.csv')
hc_only_cash_cash_test = pd.read_csv('./home-credit-default-risk/exports/hc_only-cash-cash_test_axis-24.csv')
# Revolving loans/Consumer loans
hc_only_rev_cons_train = pd.read_csv('./home-credit-default-risk/exports/hc_only-rev-cons_train_axis-24.csv')
hc_only_rev_cons_test = pd.read_csv('./home-credit-default-risk/exports/hc_only-rev-cons_test_axis-24.csv')
# hc  Revolving loans/Cash loans

app_test.head()

,SK_ID_CURR,FLAG_OWN_CAR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_BLANK,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100001,-0.717914,-0.577538,135000.0,568800.0,-0.451785,-0.238611,0.078512,-0.757568,-0.468192,...,0,0,0,0,0,1,0,0,1,0
1,100005,-0.717914,-0.577538,99000.0,222768.0,-0.671914,-0.969502,1.161210,-0.507261,-0.483339,...,0,0,0,0,0,0,0,1,0,0
2,100013,1.392925,-0.577538,202500.0,663264.0,2.943924,0.248649,0.100879,-0.923358,-0.483262,...,0,0,0,0,0,0,0,1,0,0
3,100028,-0.717914,2.192084,315000.0,1575000.0,1.511684,2.806765,0.646243,0.421966,-0.464915,...,0,0,0,0,1,0,0,0,1,0
4,100038,1.392925,0.807273,180000.0,625500.0,0.342110,0.236468,-0.864380,0.650457,-0.467215,...,0,0,0,0,0,0,0,1,0,0


In [7]:
columns = ['TARGET']
hc_br_train.drop(columns, axis=1, inplace=True)
hc_only_cash_cash_train.drop(columns, axis=1, inplace=True)
hc_only_rev_cons_train.drop(columns, axis=1, inplace=True)
#hc_only_consumer_train.drop(columns, axis=1, inplace=True)
app_train.head()

,SK_ID_CURR,TARGET,FLAG_OWN_CAR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_BLANK,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002,1.0,-0.717914,-0.577538,202500.0,406597.5,-0.166144,-0.506604,0.074115,1.597495,...,0,0,0,0,0,1,0,0,1,0
1,100003,0.0,-0.717914,-0.577538,270000.0,1293502.5,0.592667,1.600796,-1.741168,-0.222770,...,1,0,0,0,0,0,0,0,1,0
2,100004,0.0,1.392925,-0.577538,67500.0,135000.0,-1.404644,-1.091317,-0.864380,-0.716560,...,0,0,0,0,0,0,0,1,0,0
3,100006,0.0,-0.717914,-0.577538,135000.0,312682.5,0.177867,-0.652782,-1.120387,-0.707915,...,0,0,0,0,0,0,0,1,0,0
4,100007,0.0,-0.717914,-0.577538,121500.0,513000.0,-0.361746,-0.068070,0.784973,-0.901472,...,0,0,0,0,0,0,0,1,0,0


In [8]:
# trainへマージ
app_train = pd.merge(app_train, hc_br_train, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, hc_br_test, on='SK_ID_CURR', how='left')
len(app_train)

307511

In [9]:
# trainへマージ
app_train = pd.merge(app_train, hc_only_cash_cash_train, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, hc_only_cash_cash_test, on='SK_ID_CURR', how='left')
len(app_train)

307511

In [10]:
# trainへマージ
app_train = pd.merge(app_train, hc_only_rev_cons_train, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, hc_only_rev_cons_test, on='SK_ID_CURR', how='left')
len(app_train)

307511

In [11]:
# trainへマージ
#app_train = pd.merge(app_train, hc_only_consumer_train, on='SK_ID_CURR', how='left')
# testへマージ
#app_test = pd.merge(app_test, hc_only_consumer_test, on='SK_ID_CURR', how='left')
#len(app_train)

In [12]:
#num_cols = ['max_credit_day_overdue', 'max_days_credit_enddate', 'min_days_enddate_fact', 'max_amt_credit_max_overdue', 'cnt_credit_prolong_in_6month', 'sum_amt_credit_sum', 'sum_amt_credit_sum_debt', 'sum_amt_credit_sum_limit', 'sum_amt_credit_sum_overdue', 'avg_amt_annuity']

data = pd.concat([app_train, app_test], sort=False)
train = data[:len(app_train)]
test = data[len(app_test):]


y_train = train['TARGET']
X_train = app_train.copy().drop('TARGET', axis=1)
X_test = app_test.copy()

# 学習データに基づいてYao-Johnson変換を定義
#pt = PowerTransformer(method = 'yeo-johnson')
#pt.fit(X_train[num_cols])

# 変換後のデータで各列を置換
#X_train[num_cols] = pt.transform(X_train[num_cols])
#X_test[num_cols] = pt.transform(X_test[num_cols])

X_train.head()

,SK_ID_CURR,FLAG_OWN_CAR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,hc_re_cons_variance_samp_rate_interest_privileged,hc_re_cons_max_sk_dpd,hc_re_cons_min_sk_dpd,hc_re_cons_sum_sk_dpd,hc_re_cons_avg_sk_dpd,hc_re_cons_cnt_sk_dpd,hc_re_cons_stddev_sk_dpd,hc_re_cons_stddev_pop_sk_dpd,hc_re_cons_var_pop_sk_dpd,hc_re_cons_variance_samp_sk_dpd
0,100002,-0.717914,-0.577538,202500.0,406597.5,-0.166144,-0.506604,0.074115,1.597495,-0.456215,...,-0.355567,-0.084506,0.195024,0.286131,0.028207,-0.185807,-0.453571,-0.453571,-0.131680,-0.131680
1,100003,-0.717914,-0.577538,270000.0,1293502.5,0.592667,1.600796,-1.741168,-0.222770,-0.460115,...,-0.229200,-1.959827,-1.838999,-1.256639,-2.158378,0.465464,-1.009195,-1.009195,0.086914,0.086914
2,100004,1.392925,-0.577538,67500.0,135000.0,-1.404644,-1.091317,-0.864380,-0.716560,-0.453299,...,-1.204110,-0.198441,0.187722,0.059347,0.009637,-2.543880,-0.567230,-0.567230,-0.370260,-0.370260
3,100006,-0.717914,-0.577538,135000.0,312682.5,0.177867,-0.652782,-1.120387,-0.707915,-0.473217,...,0.068768,0.070916,0.850526,0.315517,0.096725,-0.733422,-0.874109,-0.874109,-0.130120,-0.130120
4,100007,-0.717914,-0.577538,121500.0,513000.0,-0.361746,-0.068070,0.784973,-0.901472,-0.473210,...,-1.542637,0.310938,0.217133,0.236815,0.158444,-0.335089,0.271508,0.271508,-0.050526,-0.050526


## 特徴量の重要度

In [13]:
dtrain = xgb.DMatrix(X_train, label=y_train)
params = {
    'booster': 'gbtree'
    ,'objective': 'binary:logistic'
    ,'silent': 1
    ,'random_state': 17
    ,'gpu_id': 0
    ,'tree_method': 'gpu_hist'
    ,'eval_metric': 'auc'
    ,'eta': 0.1
    ,'num_round': 10000
    ,'early_stopping_round': 50
    ,'max_depth': 8
    ,'subsample': 0.8
    ,'colsample_state': 71
    ,'colsample_bytree': 1.0
    ,'colsample_bylevel': 0.3
    ,'subsumple': 0.9
    ,'gamma': 0
    ,'lambda': 1
    ,'alpha': 0
    ,'min_child_weight': 1
}
num_round = 50
model = xgb.train(params, dtrain, num_round)

# 重要度の上位を出力する
fscore = model.get_score(importance_type = 'total_gain')
fscore = sorted([(k, v) for k, v in fscore.items()], key=lambda tpl: tpl[1], reverse=True)
len(fscore)

[00:46:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "early_stopping_round", "num_round", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




362

In [14]:
fscore[:20]

[('EXT_SOURCE_3', 20382.421875),
 ('EXT_SOURCE_2', 19976.486328125),
 ('EXT_SOURCE_1', 10298.873046875),
 ('CREDIT_TERM', 3669.45068359375),
 ('br_avg_days_credit_update', 3506.549560546875),
 ('OWN_CAR_AGE', 3220.7197265625),
 ('br_min_days_enddate_fact', 2366.155517578125),
 ('CREDIT_ANNUITY_RATIO', 2264.496826171875),
 ('br_min_days_credit_update', 2129.476318359375),
 ('hc_re_cons_max_rate_down_payment', 1837.64404296875),
 ('DAYS_EMPLOYED_PERCENT', 1740.58544921875),
 ('NAME_EDUCATION_TYPE_Higher education', 1316.603515625),
 ('CODE_GENDER_F', 1293.0091552734375),
 ('br_avg_days_enddate_fact', 1188.72998046875),
 ('DAYS_BIRTH', 1181.4107666015625),
 ('br_stddev_days_enddate_fact', 1155.596435546875),
 ('AMT_ANNUITY', 1094.709716796875),
 ('NAME_EDUCATION_TYPE_Secondary / secondary special', 1088.154052734375),
 ('br_max_days_credit_update', 1085.64404296875),
 ('CODE_GENDER_M', 1021.0448608398438)]

In [15]:
type(fscore)

list

In [16]:
test_columns = []
for i in range(50):
    test_columns.append(fscore[i][0])
test_columns.append('SK_ID_CURR')

In [17]:
train_columns = test_columns.copy()
train_columns.append('TARGET')

In [18]:
app_train = app_train[train_columns].copy()
app_test = app_test[test_columns].copy()
app_train.head()

,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,CREDIT_TERM,br_avg_days_credit_update,OWN_CAR_AGE,br_min_days_enddate_fact,CREDIT_ANNUITY_RATIO,br_min_days_credit_update,hc_re_cons_max_rate_down_payment,...,DAYS_LAST_PHONE_CHANGE,hc_cc_max_amt_goods_price,hc_re_cons_cnt_rate_interest_primary,hc_cc_max_amt_credit,br_min_cnt_credit_prolong,br_stddev_days_credit_enddate,hc_cc_sum_sk_dpd,hc_re_cons_cnt_rate_interest_privileged,SK_ID_CURR,TARGET
0,-2.091820,-1.317166,-2.527490,0.600237,-0.231633,19.438820,-0.077509,-0.658403,-0.347107,0.157490,...,-0.206993,-0.758897,0.545924,-0.721564,-0.636733,-0.553078,0.058061,0.545924,100002,1.0
1,-0.325247,0.565084,-1.260793,-1.588630,0.301921,15.560986,0.096564,1.868880,-0.012773,0.906781,...,0.163105,1.442642,0.543809,1.615376,0.011332,0.243727,-0.107335,0.543809,100003,0.0
2,1.243703,0.217582,0.507986,0.071863,-1.102623,26.000000,-0.701319,-0.206079,-0.706929,4.304200,...,0.178828,-0.614487,-1.534094,-0.642196,1.045890,-0.525412,0.075292,-1.534094,100004,0.0
3,0.396405,0.712794,0.659206,1.611082,-0.341181,12.614031,-0.249665,-1.416126,-0.237650,-0.620707,...,0.418304,0.755022,-0.146091,0.575474,0.387347,-0.032082,-0.107335,-0.146091,100006,0.0
4,0.286478,-1.003946,0.359715,-0.378569,-0.334401,15.119216,-0.206053,0.236368,-0.204413,-0.025661,...,-0.173128,-0.730465,-0.560231,-0.771079,0.845428,-0.164210,-0.107335,-0.560231,100007,0.0


## 学習

In [19]:
#num_cols = ['max_credit_day_overdue', 'max_days_credit_enddate', 'min_days_enddate_fact', 'max_amt_credit_max_overdue', 'cnt_credit_prolong_in_6month', 'sum_amt_credit_sum', 'sum_amt_credit_sum_debt', 'sum_amt_credit_sum_limit', 'sum_amt_credit_sum_overdue', 'avg_amt_annuity']

data = pd.concat([app_train, app_test], sort=False)
train = data[:len(app_train)]
test = data[len(app_test):]


y_train = train['TARGET']
X_train = app_train.copy().drop('TARGET', axis=1)
X_test = app_test.copy()

# 学習データに基づいてYao-Johnson変換を定義
#pt = PowerTransformer(method = 'yeo-johnson')
#pt.fit(X_train[num_cols])

# 変換後のデータで各列を置換
#X_train[num_cols] = pt.transform(X_train[num_cols])
#X_test[num_cols] = pt.transform(X_test[num_cols])

X_train.head()

,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,CREDIT_TERM,br_avg_days_credit_update,OWN_CAR_AGE,br_min_days_enddate_fact,CREDIT_ANNUITY_RATIO,br_min_days_credit_update,hc_re_cons_max_rate_down_payment,...,hc_re_cons_stddev_sk_dpd,DAYS_LAST_PHONE_CHANGE,hc_cc_max_amt_goods_price,hc_re_cons_cnt_rate_interest_primary,hc_cc_max_amt_credit,br_min_cnt_credit_prolong,br_stddev_days_credit_enddate,hc_cc_sum_sk_dpd,hc_re_cons_cnt_rate_interest_privileged,SK_ID_CURR
0,-2.091820,-1.317166,-2.527490,0.600237,-0.231633,19.438820,-0.077509,-0.658403,-0.347107,0.157490,...,-0.453571,-0.206993,-0.758897,0.545924,-0.721564,-0.636733,-0.553078,0.058061,0.545924,100002
1,-0.325247,0.565084,-1.260793,-1.588630,0.301921,15.560986,0.096564,1.868880,-0.012773,0.906781,...,-1.009195,0.163105,1.442642,0.543809,1.615376,0.011332,0.243727,-0.107335,0.543809,100003
2,1.243703,0.217582,0.507986,0.071863,-1.102623,26.000000,-0.701319,-0.206079,-0.706929,4.304200,...,-0.567230,0.178828,-0.614487,-1.534094,-0.642196,1.045890,-0.525412,0.075292,-1.534094,100004
3,0.396405,0.712794,0.659206,1.611082,-0.341181,12.614031,-0.249665,-1.416126,-0.237650,-0.620707,...,-0.874109,0.418304,0.755022,-0.146091,0.575474,0.387347,-0.032082,-0.107335,-0.146091,100006
4,0.286478,-1.003946,0.359715,-0.378569,-0.334401,15.119216,-0.206053,0.236368,-0.204413,-0.025661,...,0.271508,-0.173128,-0.730465,-0.560231,-0.771079,0.845428,-0.164210,-0.107335,-0.560231,100007


In [20]:
y_preds = []
models = []
scores = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)

categorical_features = columns

params = {
    'booster': 'gbtree'
    ,'objective': 'binary:logistic'
    ,'silent': 1
    ,'random_state': 17
    ,'gpu_id': 0
    ,'tree_method': 'gpu_hist'
    ,'eval_metric': 'auc'
    ,'eta': 0.1
    ,'num_round': 10000
    ,'early_stopping_round': 50
    ,'max_depth': 8
    ,'subsample': 0.8
    ,'colsample_state': 71
    ,'colsample_bytree': 1.0
    ,'colsample_bylevel': 0.3
    ,'subsumple': 0.9
    ,'gamma': 0
    ,'lambda': 1
    ,'alpha': 0
    ,'min_child_weight': 1
}

num_round = 50

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    dvalid = xgb.DMatrix(X_val, label=y_val)
    dtest = xgb.DMatrix(X_test)
    
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    model = xgb.train(params, dtrain, num_round, evals=watchlist)
    
    va_pred = model.predict(dvalid)
    score = log_loss(y_val, va_pred)
    oof_train[valid_index] = va_pred
    
    y_pred = model.predict(dtest) 
    
    
    #oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    #y_pred = model.predict(X_test, num_iteration=model.best_iteration)##

    y_preds.append(y_pred)
    models.append(model)
    scores.append(score)

[00:46:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "early_stopping_round", "num_round", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.72687	eval-auc:0.70534
[1]	train-auc:0.74401	eval-auc:0.71871
[2]	train-auc:0.75231	eval-auc:0.72537
[3]	train-auc:0.75528	eval-auc:0.72678
[4]	train-auc:0.76006	eval-auc:0.73013
[5]	train-auc:0.76323	eval-auc:0.73218
[6]	train-auc:0.76519	eval-auc:0.73369
[7]	train-auc:0.76768	eval-auc:0.73564
[8]	train-auc:0.76939	eval-auc:0.73621
[9]	train-auc:0.77070	eval-auc:0.73735
[10]	train-auc:0.77232	eval-auc:0.73868
[11]	train-auc:0.77358	eval-auc:0.73926
[12]	train-auc:0.77553	eval-auc:0.74055
[13]	t

[8]	train-auc:0.77001	eval-auc:0.73547
[9]	train-auc:0.77163	eval-auc:0.73625
[10]	train-auc:0.77343	eval-auc:0.73703
[11]	train-auc:0.77473	eval-auc:0.73791
[12]	train-auc:0.77625	eval-auc:0.73832
[13]	train-auc:0.77836	eval-auc:0.73892
[14]	train-auc:0.77946	eval-auc:0.73940
[15]	train-auc:0.78050	eval-auc:0.73991
[16]	train-auc:0.78162	eval-auc:0.74022
[17]	train-auc:0.78308	eval-auc:0.74126
[18]	train-auc:0.78522	eval-auc:0.74186
[19]	train-auc:0.78672	eval-auc:0.74249
[20]	train-auc:0.78819	eval-auc:0.74318
[21]	train-auc:0.78947	eval-auc:0.74375
[22]	train-auc:0.79098	eval-auc:0.74419
[23]	train-auc:0.79233	eval-auc:0.74484
[24]	train-auc:0.79348	eval-auc:0.74517
[25]	train-auc:0.79568	eval-auc:0.74595
[26]	train-auc:0.79684	eval-auc:0.74635
[27]	train-auc:0.79823	eval-auc:0.74676
[28]	train-auc:0.79931	eval-auc:0.74739
[29]	train-auc:0.80085	eval-auc:0.74799
[30]	train-auc:0.80227	eval-auc:0.74842
[31]	train-auc:0.80362	eval-auc:0.74881
[32]	train-auc:0.80493	eval-auc:0.74904
[3

In [21]:
# CSV-00
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[0]
submit_all.isnull().sum()

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_00.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [22]:
# CSV-01
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[1]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_01.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [23]:
# CSV-02
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[2]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_02.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [24]:
# CSV-03
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[3]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_03.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [25]:
# CSV-04
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[4]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_04.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [26]:
submit_all_00 = pd.read_csv('./home-credit-default-risk/submit/submit_all_00.csv')
submit_all_01 = pd.read_csv('./home-credit-default-risk/submit/submit_all_01.csv')
submit_all_02 = pd.read_csv('./home-credit-default-risk/submit/submit_all_02.csv')
submit_all_03 = pd.read_csv('./home-credit-default-risk/submit/submit_all_03.csv')
submit_all_04 = pd.read_csv('./home-credit-default-risk/submit/submit_all_04.csv')

In [27]:
submit_all = pd.DataFrame({
    'submit_all_00': submit_all_00['TARGET'].values
    ,'submit_all_01': submit_all_01['TARGET'].values
    ,'submit_all_02': submit_all_02['TARGET'].values
    ,'submit_all_03': submit_all_03['TARGET'].values
    ,'submit_all_04': submit_all_04['TARGET'].values
    })
submit_all.head()

,submit_all_00,submit_all_01,submit_all_02,submit_all_03,submit_all_04
0,0.087380,0.054248,0.056579,0.048850,0.089975
1,0.189092,0.204124,0.180347,0.173705,0.190090
2,0.014764,0.016587,0.018912,0.020479,0.025419
3,0.044754,0.033758,0.042197,0.055770,0.044345
4,0.088942,0.109080,0.123430,0.094318,0.112615


In [28]:
submit_all.corr()

,submit_all_00,submit_all_01,submit_all_02,submit_all_03,submit_all_04
submit_all_00,1.000000,0.919813,0.921256,0.921629,0.920879
submit_all_01,0.919813,1.000000,0.918150,0.922403,0.920220
submit_all_02,0.921256,0.918150,1.000000,0.923691,0.921378
submit_all_03,0.921629,0.922403,0.923691,1.000000,0.924441
submit_all_04,0.920879,0.920220,0.921378,0.924441,1.000000


In [29]:
submit_all = app_test[["SK_ID_CURR"]].copy()
submit_all['TARGET'] = (submit_all_00['TARGET'] + submit_all_01['TARGET'] + submit_all_02['TARGET'] + submit_all_03['TARGET'] + submit_all_04['TARGET'])/5
submit_all.head()

,SK_ID_CURR,TARGET
0,100001,0.067406
1,100005,0.187472
2,100013,0.019232
3,100028,0.044165
4,100038,0.105677


In [30]:
submit_all['TARGET'] = (submit_all['TARGET'] / 5)
submit_all.to_csv('./home-credit-default-risk/submit/submit_all_xgb.csv', index=False)
submit_all.head()

,SK_ID_CURR,TARGET
0,100001,0.013481
1,100005,0.037494
2,100013,0.003846
3,100028,0.008833
4,100038,0.021135


In [31]:
submit_all.count()

SK_ID_CURR    48744
TARGET        48744
dtype: int64